In [ ]:
import keyboard
import uuid
import time
from PIL import Image
from mss import mss

"""
http://www.trex-game.skipser.com/
"""

mon = {"top":300, "left":770, "width":250, "height":100} #Paintten yapıldı.
sct = mss() #Pikseller doğrultusunda iligi alanı kesip frame yapan kütüphane

i = 0
#Veri toplamak için
def record_screen(record_id, key): #Ekranı kaydedicez
    global i #Hem içerde hem dışarda kullanılabilir
    
    i += 1
    print("{}: {}".format(key, i))
    img = sct.grab(mon)
    im = Image.frombytes("RGB", img.size, img.rgb)
    im.save("./img/{}_{}_{}.png".format(key, record_id, i)) # Resmi kaydet
    #kalsörün içine img adlı bir kalsör açılmalı. Yoksa hata verir.
    
is_exit = False

def exit():
    global is_exit
    is_exit = True
    
keyboard.add_hotkey("esc", exit)  #Çıkışı çağırma fonksiyonu

record_id = uuid.uuid4()

while True:
    
    if is_exit: break

    try:
        if keyboard.is_pressed(keyboard.KEY_UP):
            record_screen(record_id, "up")
            time.sleep(0.1) #Bekeleme vermek önmeli
        elif keyboard.is_pressed(keyboard.KEY_DOWN):
            record_screen(record_id, "down")
            time.sleep(0.1)
        elif keyboard.is_pressed("right"):
            record_screen(record_id, "right") #Hiçbirşey yapamamsı gerektiği durumlarada basılacak.
            time.sleep(0.1)
    except RuntimeError: continue #Kod çakılmasını engeller.
            

In [1]:
import glob
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

imgs = glob.glob("./img_nihai/*.png") #Klasör içindeki resimleri okuma

ArithmeticError()
width = 125
height = 50

X = []
Y = []

for img in imgs: #Resimlere preprocess uygulayalım
    
    filename = os.path.basename(img)
    label = filename.split("_")[0] #Alt çizgilere göre ayırarak bir liste oluşturuyporuz
    #3 adet cloumn oluşur.
    im = np.array(Image.open(img).convert("L").resize((width, height)))
    im = im / 255
    X.append(im)    
    Y.append(label)
    
X = np.array(X)  #Traintest split içine array kabul ettiğinden array alınmalı
X = X.reshape(X.shape[0], width, height, 1) #(img , w, h , channel)

# sns.countplot(Y)

def onehot_labels(values): #Etiketleri encode ediyoruz.
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    onehot_encoder = OneHotEncoder(sparse = False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded),1) #integer encode (169,) olacağından 1 yazarak vektör haline gietiriyoruz.
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

Y = onehot_labels(Y)
train_X, test_X, train_y, test_y = train_test_split(X, Y , test_size = 0.25, random_state = 2)    

# cnn model
model = Sequential()   
model.add(Conv2D(32, kernel_size = (3,3), activation = "relu", input_shape = (width, height, 1)))
model.add(Conv2D(64, kernel_size = (3,3), activation = "relu"))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(3, activation = "softmax"))

# if os.path.exists("./trex_weight.h5"):
#     model.load_weights("trex_weight.h5")
#     print("Weights yuklendi")    

model.compile(loss = "categorical_crossentropy", optimizer = "Adam", metrics = ["accuracy"])

model.fit(train_X, train_y, epochs = 35, batch_size = 64)

score_train = model.evaluate(train_X, train_y)
print("Eğitim doğruluğu: %",score_train[1]*100)    
 #Score 0. indeksi kaybı 1. indeksi accuracy return eder
    
score_test = model.evaluate(test_X, test_y)
print("Test doğruluğu: %",score_test[1]*100)      
    
open("model_new.json","w").write(model.to_json())  #Modeli jason olarak kaydetme
model.save_weights("trex_weight_new1.h5")   
    

Epoch 1/35
2/2 [==============================] - 5s 780ms/step - loss: 3.5365 - accuracy: 0.4841
Epoch 2/35
2/2 [==============================] - 1s 720ms/step - loss: 3.8956 - accuracy: 0.3492
Epoch 3/35
2/2 [==============================] - 1s 723ms/step - loss: 1.2898 - accuracy: 0.5317
Epoch 4/35
2/2 [==============================] - 1s 650ms/step - loss: 1.2034 - accuracy: 0.4444
Epoch 5/35
2/2 [==============================] - 1s 695ms/step - loss: 0.8986 - accuracy: 0.5794
Epoch 6/35
2/2 [==============================] - 1s 688ms/step - loss: 0.7841 - accuracy: 0.6349
Epoch 7/35
2/2 [==============================] - 1s 695ms/step - loss: 0.6841 - accuracy: 0.6984
Epoch 8/35
2/2 [==============================] - 1s 633ms/step - loss: 0.6201 - accuracy: 0.7540
Epoch 9/35
2/2 [==============================] - 1s 634ms/step - loss: 0.5376 - accuracy: 0.7778
Epoch 10/35
2/2 [==============================] - 1s 628ms/step - loss: 0.4934 - accuracy: 0.8333
Epoch 11/35
2/2 [==

In [1]:
from keras.models import model_from_json
import numpy as np
from PIL import Image
import keyboard
import time
from mss import mss

mon = {"top":377, "left":715, "width":270, "height":100} #Paintten yapıldı.
sct = mss() #Pikseller doğrultusunda iligi alanı kesip frame yapan kütüphane

width = 125
height = 50

# model yükle
model = model_from_json(open("model.json","r").read())
model.load_weights("trex_weight.h5")

# down = 0, right = 1, up = 2
labels = ["Down", "Right", "Up"]

framerate_time = time.time()
counter = 0
i = 0
delay = 0.4
key_down_pressed = False
while True:
    
    img = sct.grab(mon) #Ekrandaki piksellerden görüntü alıyoum
    im = Image.frombytes("RGB", img.size, img.rgb) #Resize yapıyoruz
    im2 = np.array(im.convert("L").resize((width, height))) #array kaydediyoruz
    im2 = im2 / 255
    
    X =np.array([im2]) #Array çevirme
    X = X.reshape(X.shape[0], width, height, 1) # (a,) 1 kelnerek vektör yapışır
    r = model.predict(X) #MOdel tahmin
    
    result = np.argmax(r)
    
    # Butona basma kodları sağa gitme önemsiz (hiçbişe yapma demek)            
    if result == 0: # down = 0
        
        keyboard.press(keyboard.KEY_DOWN)
        key_down_pressed = True
        
    elif result == 2:    # up = 2
        
        if key_down_pressed:
            keyboard.release(keyboard.KEY_DOWN) #Keydown bırak

        time.sleep(delay)
        keyboard.press(keyboard.KEY_UP)
        # Havada vakit geçirme oyun özellik kodu
        if i < 1500:
            time.sleep(0.3)
        elif 1500 < i and i < 5000:
            time.sleep(0.2)
        else:
            time.sleep(0.17)
            
        keyboard.press(keyboard.KEY_DOWN)
        keyboard.release(keyboard.KEY_DOWN) #Aşağı basmayı bırak
    
    counter += 1
    
    if (time.time() - framerate_time) > 1: #Şuanki zaman - oyun başladığı zaman
        
        counter = 0
        framerate_time = time.time() #Frame time güncelle
        if i <= 1500:
            delay -= 0.003      #Frame güncellemesi
        else:
            delay -= 0.005
        if delay < 0:
            delay = 0
            
        print("---------------------")
        print("Down: {} \nRight:{} \nUp: {} \n".format(r[0][0],r[0][1],r[0][2]))
        i += 1

---------------------
Down: 0.045936379581689835 
Right:0.014584002085030079 
Up: 0.939479649066925 

---------------------
Down: 0.002517461543902755 
Right:0.9933848977088928 
Up: 0.00409762654453516 

---------------------
Down: 0.002517461543902755 
Right:0.9933848977088928 
Up: 0.00409762654453516 

---------------------
Down: 0.002517461543902755 
Right:0.9933848977088928 
Up: 0.00409762654453516 

---------------------
Down: 0.002517461543902755 
Right:0.9933848977088928 
Up: 0.00409762654453516 

---------------------
Down: 0.002517461543902755 
Right:0.9933848977088928 
Up: 0.00409762654453516 

---------------------
Down: 0.002517461543902755 
Right:0.9933848977088928 
Up: 0.00409762654453516 

---------------------
Down: 0.002517461543902755 
Right:0.9933848977088928 
Up: 0.00409762654453516 

---------------------
Down: 1.859543590398971e-05 
Right:0.0014633698156103492 
Up: 0.9985180497169495 

---------------------
Down: 0.9999914169311523 
Right:8.52110952109797e-06 
Up:

KeyboardInterrupt: 